In [6]:
from bs4 import BeautifulSoup
import pandas as pd, numpy as np
import warnings
warnings.filterwarnings('ignore')

from selenium import webdriver
from selenium.webdriver.support.ui import Select
import string, re, os, snscrape
import time
from time import sleep
import requests
from tqdm.notebook import tqdm
from urllib.parse import quote
import datetime

In [8]:
# headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}

# options = webdriver.ChromeOptions()
# driver = webdriver.Chrome('./chromedriver.exe', options=options)

# 검색 키워드 리스트
park_list = pd.read_csv('./220922_parks_concat_modify_list_modi.csv', encoding='euc-kr').iloc[:,1].to_list()

start_date = '20190601'
end_date = '20220531'

# 컬럼 리스트 생성
w_date = [] # 인플루언서 태그 있는 것 -> 날짜 가져오기
title = []
p_name = []

# url = ('https://search.naver.com/search.naver?where=view&query=' + "\"" + '%EC%9D%B8%EC%B2%9C' + "\""
#        + "%20" + quote(park_list[i]) + '&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom' + start_date + 'to'
#        + end_date + '%2Ca%3Aall&mode=normal&main_q=&st_coll=&topic_r_cat=')
# print(url)

for i in tqdm(range(len())):
    url = ('https://search.naver.com/search.naver?where=view&query=' + "\"" + '%EC%9D%B8%EC%B2%9C' + "\""
           + quote(park_list[i]) + '&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom' + start_date + 'to'
           + end_date + '%2Ca%3Aall&mode=normal&main_q=&st_coll=&topic_r_cat=')

    # 끝까지 스크롤 다운
    # options.add_argument('--headless') # 창 새로 띄우는 것 없이
    options.add_argument('--disable-gpu') # gpu 사용하지 않겠다. (괜히 오류 나니까)

    #driver.maximize_window()

    driver.get(url)

    # 스크롤 높이 가져옴
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        #끝까지 스크롤 내리기
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 대기
        time.sleep(1)

        # 스크롤 내린 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        else:
            last_height = new_height

    time.sleep(2)
    
    # 검색 키워드
    searched = driver.find_element_by_name('oquery').get_attribute("value")

    # #블로그 제목
    boxes = driver.find_elements_by_css_selector('#main_pack > section > div > div._list > panel-list > div:nth-child(1) > more-contents > div > ul > li')

    for box in boxes:
        try :
            title.append(box.find_element_by_css_selector('div.total_wrap.api_ani_send > div > a').text)
            p_name.append(searched)
            
        except :
            title.append('fail')
            p_name.append('fail')

    # 날짜
    dates = driver.find_elements_by_css_selector('#main_pack > section > div > div._list > panel-list > div:nth-child(1) > more-contents > div > ul > li')

    for date in dates:
        try :
            # 인플루언서 있는 거
            w_date.append(date.find_element_by_css_selector('div.total_wrap.api_ani_send > div > div.total_info > div.total_sub > span > span > span.etc_dsc_area > span.sub_time.sub_txt').text)
        except :
            w_date.append('fail')


driver.quit()

blog = pd.DataFrame({'Park':p_name,'Date':w_date, 'Title':title})

  0%|          | 0/10 [00:00<?, ?it/s]

MaxRetryError: HTTPConnectionPool(host='localhost', port=54086): Max retries exceeded with url: /session/a3704dc63e613a42b8d9aa301d8fd03f/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001892C4F6790>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))

In [3]:
blog = pd.DataFrame({'Date':w_date, 'Title':title, 'Park':p_name})
blog['Park'] = blog['Park'].str[4:-1]

blog.insert(0, 'Scrap_day', '')
blog['Scrap_day'] = datetime.datetime.now().date()

blog = blog.sort_values('Date').reset_index(drop=True)
blog.to_csv('./naver_blog_crawling_' + str(datetime.datetime.now().date()) + '.csv', encoding='utf-8-sig', index=False)